In [ ]:
import io
import os
import signal
import RPi.GPIO as GPIO
import time
import subprocess
import picamera
import numpy as np

from google.cloud import vision
from google.cloud.vision import types

TRIG = 11
ECHO = 12
MIN_V = 45
MAX_V = 70
MIN_T = 0.00011764705
MAX_T = 0.02352941176
TMP_FILE = 'resources/tmp.jpg'
FS = 5000       # sampling rate, Hz, must be integer
SENSE_DUR = 2   # no idea what unit this is in, may be float
SPEAK_VOL = 70
IMG_CYCLE = 20

def set_volume(percent):
    """OS call to set volume"""
    subprocess.call(['amixer', 'set', 'PCM', str.format('{}%', percent)])

def speak(string):
    """External voice program process"""
    set_volume(SPEAK_VOL)
    subprocess.call(['espeak', string])
    
# Part II: Regarding ultrasonic
def dur_to_vol(t):
    return MAX_V - ((MAX_V - MIN_V) / (MAX_T - MIN_T))*t

def sound_proc(freq):
    return subprocess.Popen(['speaker-test', '--frequency', str(freq), '--test', 'sine'])

def duration():
    """Calculated with sonic sensor"""
    GPIO.output(TRIG, 0)
    time.sleep(0.000002)
    GPIO.output(TRIG, 1)
    time.sleep(0.00001)
    GPIO.output(TRIG, 0)
    while GPIO.input(ECHO) == 0: pass
    time1 = time.time()
    while GPIO.input(ECHO) == 1: pass
    time2 = time.time()
    t = time2 - time1
    if (MIN_T < t < MAX_T):
        return t

# Part III: Regarding color
def avg_color(rgb_array):
    return np.mean(rgb_array, axis=(0, 1)).astype(int)

def color_to_freq(c):
    r, g, b = (((c[k]) >> 4) for k in range(3))  # trim 4 bit from each value
    return ((r << 8) + (g << 4) + b)

def main():
    with picamera.PiCamera() as camera:
        camera.resolution = (320, 240)  # Scale down resolution
        try:
            # Google Vision setup
            os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "api-key.json"
            client = vision.ImageAnnotatorClient()  # Instantiates a client

            # Sonic setup
            GPIO.setmode(GPIO.BOARD)
            GPIO.setup(TRIG, GPIO.OUT)
            GPIO.setup(ECHO, GPIO.IN)

            # Color setup
            rgb_array = np.empty((240, 320, 3), dtype=np.uint8)

            # Turn on the sound
            set_volume(MIN_V)
            proc = sound_proc(440)

            counter = 0  # To keep count for Google Vision
            while True:
                counter += 1
                
                # Image processing (rare)
                if counter == IMG_CYCLE:
                    counter = 0

                    # Capture the image to file...
                    camera.capture(TMP_FILE)
                    # ... just to load it back to memory
                    with io.open(TMP_FILE, 'rb') as image_file:
                        content = image_file.read()
                    jpg = types.Image(content=content)

                    # Process the image on the cloud
                    response = client.label_detection(image=jpg)
                    labels = response.label_annotations

                    for label in labels[:2]:
                        print(label.description, end=' ')
                        proc.kill()
                        speak(label.description)

                # Audio sensing (common)
                else:
                    # Sonic part
                    t = duration()  # Gather distance
                    if t is not None:
                        volume = dur_to_vol(t)
                        print("{:.2f}".format(volume), end=' ')
                    else:
                        volume = MIN_V
                    set_volume(volume)

                    # Color:
                    camera.capture(rgb_array, 'rgb')
                    freq = color_to_freq(avg_color(rgb_array))
                    print(freq, end=' ')
                    
                    # Play the sound
                    try:
                        proc.kill()  # Kill the old one first
                    except e:
                        print(e)
                    proc = sound_proc(freq)
                    time.sleep(0.05)

        except Exception as e:
            print(e)
        finally:
            GPIO.cleanup()
            proc.kill()
            
main()

1331 68.00 256 

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "<ipython-input-1-9ca54e504de0>", line 120, in main
    camera.capture(rgb_array, 'rgb')
  File "/srv/pl-app/lib/python3.5/site-packages/picamera/camera.py", line 1421, in capture
    if not encoder.wait(self.CAPTURE_TIMEOUT):
  File "/srv/pl-app/lib/python3.5/site-packages/picamera/encoders.py", line 393, in wait
    result = self.event.wait(timeout)
  File "/usr/lib/python3.5/threading.py", line 549, in wait
    signaled = self._cond.wait(timeout)
  File "/usr/lib/python3.5/threading.py", line 297, in wait
    gotit = waiter.acquire(True, timeout)
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/srv/pl-app/lib/python3.5/site-packages/IPython/core/interactiveshell.py", line 2862, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-1-9ca54e504de0>", line 137, in <module>
    main()
  File "<ipython-input-1-9ca54e504de0>", line

TypeError: Can't convert 'list' object to str implicitly

ERROR:tornado.general:Uncaught exception, closing connection.
Traceback (most recent call last):
  File "/srv/pl-app/lib/python3.5/site-packages/zmq/eventloop/zmqstream.py", line 414, in _run_callback
    callback(*args, **kwargs)
  File "/srv/pl-app/lib/python3.5/site-packages/tornado/stack_context.py", line 277, in null_wrapper
    return fn(*args, **kwargs)
  File "/srv/pl-app/lib/python3.5/site-packages/ipykernel/kernelbase.py", line 283, in dispatcher
    return self.dispatch_shell(stream, msg)
  File "/srv/pl-app/lib/python3.5/site-packages/ipykernel/kernelbase.py", line 235, in dispatch_shell
    handler(stream, idents, msg)
  File "/srv/pl-app/lib/python3.5/site-packages/ipykernel/kernelbase.py", line 421, in execute_request
    self._abort_queues()
  File "/srv/pl-app/lib/python3.5/site-packages/ipykernel/kernelbase.py", line 637, in _abort_queues
    self._abort_queue(stream)
  File "/srv/pl-app/lib/python3.5/site-packages/ipykernel/kernelbase.py", line 662, in _abort_queue
 